In [32]:
import pandas as pd
import numpy as np
import datetime
import re
import os
import matplotlib.pyplot as plt
from scipy.stats import shapiro, variation
import seaborn as sns

%matplotlib inline

In [56]:
os.chdir(r'\\tsclient\X\УМиР\Аманов\Adhoc\Оле_талоны')

In [34]:
oborot = pd.read_excel('Оборот_март.xlsx', skiprows=2)
skiper = oborot.shape[0] - [i for i, j in enumerate(oborot.isnull().all(axis=1)) if j is True][0]
oborot = pd.read_excel('Оборот_март.xlsx', skiprows=2, skipfooter=skiper)
cheki = pd.read_excel('Чеки_март.xlsx', skiprows=2)
skiper = cheki.shape[0] - [i for i, j in enumerate(cheki.isnull().all(axis=1)) if j is True][0]
cheki = pd.read_excel('Чеки_март.xlsx', skiprows=2, skipfooter=skiper)

In [35]:
oborot = oborot.rename(columns={'Unnamed: 0': 'Филиал'})
oborot = oborot.iloc[:, :6]
oborot['Филиал'] = oborot['Филиал'].fillna(method='ffill')

cheki = cheki.rename(columns={'Unnamed: 0': 'Филиал'})
cheki = cheki.iloc[:, :6]
cheki['Филиал'] = cheki['Филиал'].fillna(method='ffill')

oborot = oborot[oborot['Клиенты:'] == 'Итого оборот:']
cheki = cheki[cheki['Клиенты:'] == 'Итого оборот:']

oborot.drop('Клиенты:', axis=1, inplace=True)
cheki.drop('Клиенты:', axis=1, inplace=True)

oborot = oborot.reset_index(drop=True)
cheki = cheki.reset_index(drop=True)


all_df = oborot.merge(cheki, on='Филиал')

In [39]:
for column in all_df.columns[:5]:
    for column2 in all_df.columns[-4:]:
        if column[:-2] == column2[:-2]:
            x = column
            y = column2
            all_df[column[:-1]+'_z'] = all_df[x] / all_df[y]

In [41]:
proekt = pd.read_excel('Проект_Влияние кл-о на продажи.xlsx')

In [45]:
filials = proekt['Офис'][2:33].to_list()

In [47]:
filials = set(filials)

In [48]:
filials_all_df = set(all_df['Филиал'].to_list())

In [53]:
filials_all_df - filials

set()

In [54]:
filials - filials_all_df

set()

In [55]:
all_df.to_excel('Обработано.xlsx')

In [68]:
os.chdir(r'\\tsclient\X\УМиР\Аманов\Adhoc\Оле_талоны\rfm')

In [75]:
for file in os.listdir():
    if 'rfm' in file.lower() and '$' not in file and 'до' not in file.lower():
        rfm_week = pd.read_excel(file, skiprows=2, sheet_name='RFM подробный')
        rfm_week = rfm_week[['ФИО клиента', 'Место последнего посещения']]
        for file2 in os.listdir():
            if ''.join([i for i in file2 if i.isdigit()]) in file and 'rfm' in file2.lower():
                rfm_half = pd.read_excel(file2, skiprows=2, sheet_name='RFM подробный')
                rfm_half = rfm_half[['ФИО клиента', 'Место последнего посещения', 'Количество операций за период']]
                rfm_week.dropna(subset=['ФИО клиента'], inplace=True)
                rfm_week.drop(rfm_week[rfm_week['ФИО клиента'] == ' Без паспорта'].index, inplace=True)
                rfm_week = rfm_week.merge(rfm_half, how='left', on=['ФИО клиента', 'Место последнего посещения'])
                rfm_week = rfm_week.fillna(0)

                def count_if(arr):
                    return len([i for i in arr if i > 0])/len(arr)

                rfm_week_filials = rfm_week.groupby('Место последнего посещения').agg({'Количество операций за период': count_if})
                rfm_week_filials.to_excel(f'Показатели_{file}.xlsx')
                print(file, rfm_week[rfm_week['Количество операций за период']>0].shape[0] / rfm_week.shape[0])

RFM анализ01032021-07032021.xlsx 0.5163531423685336
RFM анализ08032021-14032021.xlsx 0.4994009367171332
RFM анализ15032021-21032021.xlsx 0.4658200486355039
RFM анализ22032021-28032021.xlsx 0.47897851136717534


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\$RFM анализ_до22032021.xlsx'